In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from vae_methods import Sampling, sequencify
import tensorflow_addons as tfa
from sklearn.model_selection import train_test_split

In [2]:
data_path = "../SWaT_Data"
with open(f"{data_path}/scalerX_0.pkl", "rb") as f:
    scaler = pickle.load(f)

df = pd.read_csv(f"{data_path}/SWaT_Dataset_Attack_v0.csv")

# Relabel
df.replace(["A ttack", "Attack", "Normal"], [1, 1, 0], inplace=True)

# df

In [3]:
df.columns

Index([' Timestamp', 'FIT101', 'LIT101', ' MV101', 'P101', 'P102', ' AIT201',
       'AIT202', 'AIT203', 'FIT201', ' MV201', ' P201', ' P202', 'P203',
       ' P204', 'P205', 'P206', 'DPIT301', 'FIT301', 'LIT301', 'MV301',
       'MV302', ' MV303', 'MV304', 'P301', 'P302', 'AIT401', 'AIT402',
       'FIT401', 'LIT401', 'P401', 'P402', 'P403', 'P404', 'UV401', 'AIT501',
       'AIT502', 'AIT503', 'AIT504', 'FIT501', 'FIT502', 'FIT503', 'FIT504',
       'P501', 'P502', 'PIT501', 'PIT502', 'PIT503', 'FIT601', 'P601', 'P602',
       'P603', 'Normal/Attack', 'Unnamed: 53', 'Unnamed: 54'],
      dtype='object')

In [3]:
df_data = df.drop(columns=[df.columns[0], df.columns[-1], df.columns[-2], df.columns[-3]]).dropna()

In [4]:
df_attack = scaler.transform(df_data)
labels = df['Normal/Attack']

In [6]:
window = 5
x_seq = sequencify(df_attack, window)


In [10]:
def sequencify_y(arr, window):
    
    shape = (arr.shape[0] - window + 1, window,)
    
    # strides for processor to know how many bytes to skip / stride over to read from memory
    strides = (arr.strides[0],) + arr.strides
    
    return np.lib.stride_tricks.as_strided(arr, shape=shape, strides=strides)

y_seq = sequencify_y(labels.to_numpy(), window)

In [19]:
# labels are 0 or 1. can be changed to be the average over the axis rather than max
y_labels = np.amax(y_seq, axis=1)

In [13]:
labels[labels == 1]

1754      1
1755      1
1756      1
1757      1
1758      1
         ..
445186    1
445187    1
445188    1
445189    1
445190    1
Name: Normal/Attack, Length: 54621, dtype: int64

In [14]:
y_seq[1750:1760]

array([[0, 0, 0, 0, 1],
       [0, 0, 0, 1, 1],
       [0, 0, 1, 1, 1],
       [0, 1, 1, 1, 1],
       [1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1]])

In [17]:
y_seq.shape

(449915, 5)

In [20]:
x_seq.shape

(449915, 5, 51)

In [16]:
np.amax(y_seq, axis=1).shape

(449915,)

In [18]:
np.amax(y_seq, axis=1)[1745:1755]

array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1])

In [21]:
x_train, x_test, y_train, y_test = train_test_split(x_seq, y_labels, test_size=0.4, random_state=0)

In [22]:
x_train.shape

(269949, 5, 51)

In [23]:
y_train.shape

(269949,)

In [24]:
x_test.shape

(179966, 5, 51)

In [25]:
y_test.shape

(179966,)

In [26]:
del df_data
del df
del labels
del df_attack

In [27]:
input_dim = x_train.shape[1:]

In [31]:
input_dim

(5, 51)

In [32]:
window, input_dim = input_dim

In [28]:
encoder = tf.keras.models.load_model('LSTM_VAE_Encoder_100422.h5', custom_objects={"Sampling": Sampling})

In [29]:
encoder_checkpoint_path = "LSTM_VAE_Encoder_100422_weights.ckpt"
encoder.save_weights(encoder_checkpoint_path)

In [30]:
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=11,
    restore_best_weights=True,
)

reduce_lr_calllback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.15,
    patience=5,
    verbose=0,
    mode='auto',
    min_delta=0.0001,
    cooldown=0,
    min_lr=0
)

In [35]:
def create_classifier(encoder, input_dim):
    
    inputs = tf.keras.Input(shape=(input_dim))
    z_mean, z_logvar, z = encoder(inputs, training=False)
    latent = tf.keras.layers.Concatenate()([z_mean, z_logvar])
    dense1 = tf.keras.layers.Dense(30, activation='tanh')(latent)
    dense2 = tf.keras.layers.Dense(15, activation='relu')(dense1)
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(dense2)
    
    model = tf.keras.Model(inputs, outputs, name='Classifier')
    
    return model

In [38]:
latent_dim = input_dim - 6

strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    inputs_encoder_vae = tf.keras.Input(shape=(window, input_dim))
        
    lstm_encoder = tf.keras.layers.LSTM(input_dim, activation='tanh')(inputs_encoder_vae)

    dense1_vae = tf.keras.layers.Dense(input_dim - 5, activation='relu')(lstm_encoder)

    z_mean = tf.keras.layers.Dense(latent_dim, name="z_mean")(dense1_vae)
    z_log_var = tf.keras.layers.Dense(latent_dim, name="z_log_var")(dense1_vae)
    z = Sampling()([z_mean, z_log_var])

    encoder = tf.keras.Model(inputs=inputs_encoder_vae,
                            outputs=[z_mean, z_log_var, z],
                            name='LSTM_VAE_encoder')
    
    encoder.load_weights(encoder_checkpoint_path)
    
    encoder.trainable = False
        return model
    
    model = create_classifier(encoder, (window, input_dim))
    
#     inputs = tf.keras.Input(shape=(input_dim,))
#     z_mean, z_logvar, z = encoder(inputs, training=False)
#     latent = tf.keras.layers.Concatenate()([z_mean, z_logvar])
#     dense1 = tf.keras.layers.Dense(30, activation='tanh')(latent)
#     dense2 = tf.keras.layers.Dense(15, activation='relu')(dense1)
#     outputs = tf.keras.layers.Dense(1, activation='sigmoid')(dense2)
#     outputs = tf.keras.layers.Dense(1, activation='relu')(dense2)
    
#     model = tf.keras.Model(inputs, outputs, name='Classifier')
    
    model.compile(loss=tfa.losses.SigmoidFocalCrossEntropy(from_logits=False),
                  optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, clipnorm=1),
                 metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
#                   tf.keras.losses.BinaryCrossentropy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


In [39]:
hist = model.fit(x_train, y_train,
                 epochs=100,
                 validation_split=0.15,
                 batch_size=128,
                 callbacks=[early_stopping_callback, reduce_lr_calllback])

Epoch 1/100
INFO:tensorflow:batch_all_reduce: 6 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 6 all-reduces with algorithm = nccl, num_packs = 1
1793/1793 [==============================] - 23s 8ms/step - loss: 0.0160 - accuracy: 0.9524 - precision_2: 0.9894 - recall_2: 0.6158 - val_loss: 0.0140 - val_accuracy: 0.9597 - val_precision_2: 0.9861 - val_recall_2: 0.6779
Epoch 2/100
1793/1793 [==============================] - 12s 7ms/step - loss: 0.0134 - accuracy: 0.9595 - precision_2: 0.9922 - recall_2: 0.6724 - val_loss: 0.0135 - val_accuracy: 0.9609 - val_precision_2: 0.9955 - val_recall_2: 0.6808
Epoch 3/100
1793/1793 [==============================] - 12s 7ms/step - loss: 0.0126 - accuracy: 0.9611 - precision_2: 0.9920 - recall_2: 0.6859 - val_loss: 0.0121 - val_accuracy: 0.9626 - val_precision_2: 0.9843 - val_recall_2: 0.7032
Epoch 4/100
1793/1793 [==============================] - 12s 7ms/step - loss: 0.0119 - accuracy: 0.9622 - precision_2: 0.9

In [40]:
model.evaluate(x_test, y_test)

5624/5624 [==============================] - 25s 4ms/step - loss: 0.0072 - accuracy: 0.9725 - precision_2: 0.9868 - recall_2: 0.7846


[0.007156592793762684,
 0.9725003838539124,
 0.9867922067642212,
 0.7845500707626343]

In [41]:
model.save("Classifier2_LSTM_VAE_100422_Freeze.h5")

In [42]:
encoder.trainable = True

In [43]:
hist2 = model.fit(x_train, y_train,
                 epochs=100,
                 validation_split=0.15,
                 batch_size=128,
                 callbacks=[early_stopping_callback, reduce_lr_calllback])

Epoch 1/100
1793/1793 [==============================] - 12s 7ms/step - loss: 0.0070 - accuracy: 0.9725 - precision_2: 0.9880 - recall_2: 0.7840 - val_loss: 0.0070 - val_accuracy: 0.9730 - val_precision_2: 0.9860 - val_recall_2: 0.7888
Epoch 2/100
1793/1793 [==============================] - 12s 7ms/step - loss: 0.0070 - accuracy: 0.9725 - precision_2: 0.9880 - recall_2: 0.7840 - val_loss: 0.0070 - val_accuracy: 0.9730 - val_precision_2: 0.9860 - val_recall_2: 0.7888
Epoch 3/100
1793/1793 [==============================] - 12s 7ms/step - loss: 0.0070 - accuracy: 0.9725 - precision_2: 0.9880 - recall_2: 0.7841 - val_loss: 0.0070 - val_accuracy: 0.9730 - val_precision_2: 0.9860 - val_recall_2: 0.7888
Epoch 4/100
1793/1793 [==============================] - 12s 7ms/step - loss: 0.0070 - accuracy: 0.9725 - precision_2: 0.9879 - recall_2: 0.7842 - val_loss: 0.0070 - val_accuracy: 0.9730 - val_precision_2: 0.9860 - val_recall_2: 0.7888
Epoch 5/100
1793/1793 [==============================] -

In [44]:
model.evaluate(x_test, y_test)

5624/5624 [==============================] - 25s 4ms/step - loss: 0.0072 - accuracy: 0.9725 - precision_2: 0.9868 - recall_2: 0.7846


[0.007156273815780878,
 0.9725003838539124,
 0.9867922067642212,
 0.7845500707626343]

In [45]:
model.save("Classifier2_LSTM_VAE_100422_FineTuned.h5")